In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torchtext

from fastai.text.all import *
from torchtext.data import get_tokenizer   # for TOKENIZER
from collections import Counter       # for NUMERICALIZATION 

## Data PreProcessing

In [ ]:
# READ THE CSV FILES
train_df = pd.read_csv('../input/sentiment-analysis-on-movie-reviews/train.tsv.zip', sep='\t')
test_df = pd.read_csv('../input/sentiment-analysis-on-movie-reviews/test.tsv.zip', sep='\t')

In [ ]:
train_df.info()

In [ ]:
test_df.head()

In [ ]:
train_df.shape, test_df.shape

## Data Processing

Since it is a text data so data processing is required to feet it into the text classifier learner.

### TOkenization and NUmerclization

In [ ]:
# GET TOKENIZER
tokenizer = get_tokenizer("basic_english")

# GET COUNTER
token_counter = Counter()

In [ ]:
# count the tokens
for curr_text in train_df['Phrase']:
    
    # tokenize the phrase
    tokens = tokenizer(curr_text)
    
    # count each token
    token_counter.update(tokens)

In [ ]:
token_counter.most_common(n=10)

### CReate Vocab

In [ ]:
# create the vocabulary for model (make_vocab FROM FASTAI)
sentiments_vocab = make_vocab(token_counter)    

## DATABLOCKS AND DATALOADERS

In [ ]:
train_df.columns

In [ ]:
# CREATE TEXT DATABLOCK
sentiments_db = DataBlock(blocks = (TextBlock.from_df('Phrase', seq_len=128, vocab = sentiments_vocab),
                                       CategoryBlock),
                                    get_x = ColReader('text'),
                                    get_y = ColReader('Sentiment'))

In [ ]:
# CREATE DATALOADER
dls = sentiments_db.dataloaders(train_df, bs=32)

In [ ]:
len(dls.train_ds), len(dls.valid_ds)

In [ ]:
# SHOW BATCH
dls.show_batch()

## Training

### Learner

In [ ]:
# CREATE LEARNER
learn = text_classifier_learner(dls, AWD_LSTM, metrics=[accuracy])

In [ ]:
# find best learning rate
learn.lr_find()

### Fine - Tuning

In [ ]:
# fine tune the model
learn.fine_tune(5,base_lr = 9e-3)

In [ ]:
learn.save('finetuned')

## INFERENCE

In [ ]:
# LOAD THE MODEL
learn = learn.load('finetuned')

### Make Predictions

In [ ]:
# load the test dataloader
dl = learn.dls.test_dl(test_df['Phrase'])
dl

In [ ]:
# GET PREDICTIONS (returns a tuple)
preds = learn.get_preds(dl=dl)  
preds

In [ ]:
print("All records predictions are: \n", preds[0])   # first value of tuple (all rows)
print("\n Prediction of First recrod: \n",preds[0][0])    # first tensor of first value

In [ ]:
# INDEX OF MAX VALUE ACROSS ALL PREDICTIONS
preds[0].argmax(dim=-1)

In [ ]:
len(preds[0].argmax(dim=-1))

### Create Submission File

In [ ]:
# CREATE EMPTY DATAFRAME
data = {'PhraseId':[], 
        'Sentiment':[]}
submission = pd.DataFrame(data)

In [ ]:
# FILL THE SUBMISSIONS
submission['PhraseId'] = test_df['PhraseId']
submission['Sentiment'] = preds[0].argmax(dim=-1)

In [ ]:
submission.head(), submission.shape

In [ ]:
submission.to_csv('Submission_finetuned_68acc.csv', index=False)